In [ ]:
import geemap.core as geemap
import ee

def mask_s2_clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

ee.Authenticate()
ee.Initialize(project='flowers-spaceapps')

# Location/zoom: 241.611637, 34.738262, 14

dataset_flowers = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2019-03-01', '2019-03-31')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

dataset_winter = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2019-11-01', '2019-11-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization_RGB = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

visualization_NIR = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B8'],
}

m = geemap.Map()
m.set_center(241.611637, 34.738262, 14)
m.add_layer(dataset_flowers.mean(), visualization_RGB, 'RGB')
m


In [ ]:
n = geemap.Map()
n.set_center(241.611637, 34.738262, 14)
n.add_layer(dataset_flowers.mean(), visualization_NIR, 'NIR')

n